In [75]:
import requests
from bs4 import BeautifulSoup
import os

In [81]:
def extract_links_from_page(url):
    """
    Extracts all hyperlinks from a webpage.

    Parameters:
        url (str): The URL of the webpage to scrape.

    Returns:
        list: A list of URLs found on the page.
    """
    try:
        # Make an HTTP request to fetch the page content
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the page: {e}")
        return []

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all <a> tags with an href attribute
    links = [requests.compat.urljoin(url, link['href']) for link in soup.find_all('a', href=True)]

    return links

visited_links = {}

def fetch_files_from_page(url, output_folder="downloads", file_extensions=("pdf", "doc", "docx"), filter_function=None):
    """
    Fetches all files with specified extensions from a given URL and saves them locally.

    Parameters:
        url (str): The URL of the webpage to scrape.
        output_folder (str): The folder to save the downloaded files. Defaults to "downloads".
        file_extensions (tuple): File extensions to download. Defaults to ("pdf", "doc", "docx").
        filter_function (callable): A function that takes a file name as input and returns True if the file should be downloaded, False otherwise. Defaults to None.

    Returns:
        list: List of downloaded file paths.
    """
    # Dictionary to track visited links and avoid duplicates
    

    # Get all links from the page
    all_links = extract_links_from_page(url)
    file_links = []

    for file_url in all_links:
        # Skip if the link has already been processed
        if file_url in visited_links:
            continue

        # Mark the link as visited
        visited_links[file_url] = True

        # Check file extension
        if file_url.split('.')[-1].lower() in file_extensions:
            file_links.append(file_url)

    # Ensure output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    downloaded_files = []

    # Download each file
    for file_url in file_links:
        try:
            file_response = requests.get(file_url, stream=True)
            file_response.raise_for_status()

            # Extract the file name
            file_name = os.path.basename(file_url)

            # Apply the filter function if provided
            if filter_function and not filter_function(file_name):
                print(f"Skipped: {file_name}")
                continue

            output_path = os.path.join(output_folder, file_name)

            # Save the file locally
            with open(output_path, 'wb') as f:
                for chunk in file_response.iter_content(chunk_size=8192):
                    f.write(chunk)

            downloaded_files.append(output_path)
            print(f"Downloaded: {file_name}")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download {file_url}: {e}")

    return downloaded_files

# Centrale

In [77]:
baselink = "https://www.concours-centrale-supelec.fr/CentraleSupelec"
annee = [i for i in range(1998,2025)]
filiere = ["MP","PC","MPI","PSI","TSI"]
sujet = ["","sujets"]

In [ ]:
def filtre(nom):
    liste = ["DR","rap","Admissibilite","Admission" , "XX"] +  filiere
    for e in liste:
        if e in nom:
            return False
    return True

In [79]:
for f in filiere:
    for a in annee:
        for sujets in sujet:
            url = f"{baselink}/{a}/{f}/{sujets}"
            fetch_files_from_page(url, output_folder=f"data/centrale/{f}/{a}", filter_function=filtre)

['https://www.concours-centrale-supelec.fr', 'https://acrobat.adobe.com/fr/fr/acrobat/pdf-reader.html', 'https://www.concours-centrale-supelec.fr/CentraleSupelec/SujetsOral/MP', 'https://www.concours-centrale-supelec.fr/CentraleSupelec/SujetsOral/MPI', 'https://www.concours-centrale-supelec.fr/CentraleSupelec/SujetsOral/PC', 'https://www.concours-centrale-supelec.fr/CentraleSupelec/SujetsOral/PSI', 'https://www.concours-centrale-supelec.fr/CentraleSupelec/SujetsOral/TSI', 'https://www.concours-centrale-supelec.fr/CentraleSupelec/2024/MP/rapCS2024MP.pdf', 'https://www.concours-centrale-supelec.fr/CentraleSupelec/2024/MP', 'https://www.concours-centrale-supelec.fr/CentraleSupelec/2024/MPI/rapCS2024MPI.pdf', 'https://www.concours-centrale-supelec.fr/CentraleSupelec/2024/MPI', 'https://www.concours-centrale-supelec.fr/CentraleSupelec/2024/PC/rapCS2024PC.pdf', 'https://www.concours-centrale-supelec.fr/CentraleSupelec/2024/PC', 'https://www.concours-centrale-supelec.fr/CentraleSupelec/2024/P

In [83]:
def supprimer_fichiers_contenant_recursivement(repertoire, chaine_a_chercher):
  """
  Supprime tous les fichiers d'un répertoire et de ses sous-répertoires
  dont le nom contient une chaîne de caractères spécifique.

  Args:
    repertoire: Le chemin du répertoire où commencer la recherche.
    chaine_a_chercher: La chaîne de caractères à chercher dans les noms de fichiers.
  """

  for root, dirs, files in os.walk(repertoire):
    for file in files:
      if chaine_a_chercher in file:
        chemin_complet = os.path.join(root, file)
        os.remove(chemin_complet)
        print(f"Fichier supprimé : {chemin_complet}")


In [89]:
supprimer_fichiers_contenant_recursivement(repertoire= "data", chaine_a_chercher = "TSI")

Fichier supprimé : data\centrale\TSI\2016\TSIe-Info.pdf
Fichier supprimé : data\centrale\TSI\2016\TSIe-LVen.pdf
Fichier supprimé : data\centrale\TSI\2016\TSIe-Mat1.pdf
Fichier supprimé : data\centrale\TSI\2016\TSIe-Mat2.pdf
Fichier supprimé : data\centrale\TSI\2016\TSIe-PhC1.pdf
Fichier supprimé : data\centrale\TSI\2016\TSIe-PhC2.pdf
Fichier supprimé : data\centrale\TSI\2016\TSIe-Reda.pdf
Fichier supprimé : data\centrale\TSI\2016\TSIe-ScIn.pdf
Fichier supprimé : data\centrale\TSI\2017\TSIe-Info.pdf
Fichier supprimé : data\centrale\TSI\2017\TSIe-LVen.pdf
Fichier supprimé : data\centrale\TSI\2017\TSIe-Mat1.pdf
Fichier supprimé : data\centrale\TSI\2017\TSIe-Mat2.pdf
Fichier supprimé : data\centrale\TSI\2017\TSIe-PhC1.pdf
Fichier supprimé : data\centrale\TSI\2017\TSIe-PhC2.pdf
Fichier supprimé : data\centrale\TSI\2017\TSIe-Reda.pdf
Fichier supprimé : data\centrale\TSI\2017\TSIe-ScIn.pdf
Fichier supprimé : data\centrale\TSI\2018\TSIe-Info.pdf
Fichier supprimé : data\centrale\TSI\2018\TSIe-L

In [ ]:
import re
from PyPDF2 import PdfReader

def extraire_titres_et_questions(fichier_pdf):
    """
    Extrait les titres et numéros de questions d'un fichier PDF.

    Args:
        fichier_pdf (str): Chemin vers le fichier PDF.

    Returns:
        list: Liste des questions.
    """

    with open(fichier_pdf, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        pages = pdf_reader.pages

        resultats = []
        for page in pages:
            texte_page = page.extract_text()
            pattern = r"Q\s*(\d+[A-Z]?\d*\)?)\.?\s+(.*)" 
            matches = re.findall(pattern, texte_page, re.MULTILINE)
            
            for match in matches:
                if match:
                    numero_question, titre = match
                    resultats.append((numero_question.strip()))

        return resultats

# Exemple d'utilisation :
fichier = r"J:\01 Fichier\05 Code\99 - Autre\02 Sujetdex\data\centrale\MP\2024\M064.pdf"
resultats = extraire_titres_et_questions(fichier)
print(resultats)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']
